In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import requests
import pandas as pd
import sys
import re
from dateutil.parser import parse
from datetime import datetime, timedelta
import os
import xmltodict
import pprint
import json
from joblib import Parallel, delayed

In [ ]:
### LIB

import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
import gc
import pandas as pd
from dateutil.parser import *
import requests
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
import re
from io import StringIO
from calendar import monthrange
import yt.wrapper as yt
import json
from datetime import datetime, timedelta
import operator
import requests, os, sys, pymysql,time
from statsmodels.stats import multitest
from collections import namedtuple
import yt.transfer_manager.client as tm

try:
    from vault_client.instances import Production as VaultClient
    client = VaultClient(decode_files=True)
    secret_uuid = "sec-01e2dyrwvwrmnk1r2q7rnyejcy"
    tokens = client.get_version(secret_uuid)
    clickhouse_token = tokens["value"]["clickhouse_token"]
    metrika_password = tokens["value"]["metrika"]
    crm_password = tokens["value"]["crm"]
    startrek_token = tokens["value"]["startrek"]
    grafana_token = tokens["value"]["grafana_token"]
    mdb_auth_token = tokens["value"]["mdb_auth"]
except Exception:
    clickhouse_token = sys.argv[1]
    metrika_password = sys.argv[2]
    crm_password = sys.argv[3]

yt.config["proxy"]["url"] = "hahn"
yt.config["token"] = clickhouse_token
cluster = "hahn"
alias = "*cloud_analytics"

def raw_execute_yt_query(query, timeout=600):
    token = clickhouse_token
    proxy = "http://{}.yt.yandex.net".format(cluster)
    s = requests.Session()
    url = "{proxy}/query?database={alias}&password={token}&enable_optimize_predicate_expression=0".format(proxy=proxy,
                                                                                            alias=alias, token=token)
    resp = s.post(url, data=query, timeout=timeout)
    resp.raise_for_status()
    rows = resp.text.strip().split('\n')
    return rows

yt.config["proxy"]["url"] = "hahn"
yt.config["token"] = clickhouse_token
cluster = "hahn"
alias = "*cloud_analytics"

def raw_execute_yt_query(query, timeout=600):
    token = clickhouse_token
    proxy = "http://{}.yt.yandex.net".format(cluster)
    s = requests.Session()
    url = "{proxy}/query?database={alias}&password={token}&enable_optimize_predicate_expression=0".format(proxy=proxy,
                                                                                            alias=alias, token=token)
    resp = s.post(url, data=query, timeout=timeout)
    resp.raise_for_status()
    rows = resp.text.strip().split('\n')
    return rows

def raw_grafana_execute_query(query):
    url = 'https://{host}:8443/?database={db}&query={query}'.format(
        host='sas-tt9078df91ipro7e.db.yandex.net',
        db='cloud_analytics',
        query=query)
    auth = {
        'X-ClickHouse-User': 'lunindv',
        'X-ClickHouse-Key': grafana_token,
    }
    res = requests.get(
        url,
        headers=auth,
        verify='/usr/local/share/ca-certificates/Yandex/YandexInternalRootCA.crt')
    res.raise_for_status()
    rows = res.text.strip().split('\n')
    return rows


def raw_chyt_execute_any_query(query, request_func, columns = None):
    i = 0
    while True:
        try:
            result = request_func(query=query)
            if columns is None:
                users = pd.DataFrame([row.split('\t') for row in result[1:]], columns = result[0].split('\t'))
            else:
                users = pd.DataFrame([row.split('\t') for row in result], columns=columns)
            return users
        except Exception as err:
            print(err)
            i += 1
            if i > 5:
                print('Break Excecution')
                break


def update_automatically_types(df_raw):
    df = df_raw.copy()
    for column in df.columns:
        if ("id" in column and "paid" not in column) or \
        (len(df[column].replace(np.nan, "").max()) > 9 and
                              ("." not in df[column].replace(np.nan, "").max())):
            continue
        try:
            df[column] = df[column].astype(int)
            continue
        except Exception:
            pass
        try:
            df[column] = df[column].astype(float)
            continue
        except Exception:
            pass
        try:
            df[column] = df[column].apply(lambda x: json.loads(x))
            continue
        except Exception:
            pass
        try:
            df[column] = df[column].apply(lambda x: json.loads(x.replace("'", '"')))
            continue
        except Exception:
            pass
    return df


def execute_query_body(query, request_func, columns = None):
    df = raw_chyt_execute_any_query(query, request_func, columns)
    df = df.replace('\\N', np.NaN)
    df = update_automatically_types(df)
    if "email" in df.columns:
        df["email"] = df["email"].apply(lambda x: works_with_emails(x))
    return df


def execute_query(query, columns = None):
    """Execute query, returns pandas dataframe with result
    :param query: query to execute on cluster
    :param columns: name of dataframe columns
    :return: pandas dataframe, the result of query
    """
    return execute_query_body(query, raw_execute_yt_query, columns)


def works_with_emails(mail_):
    """mail processing
    :param mail_: mail string
    :return: processed string
    """
    mail_parts = str(mail_).split('@')
    if len(mail_parts) > 1:
        if 'yandex.' in mail_parts[1].lower() or 'ya.' in mail_parts[1].lower():
            domain = 'yandex.ru'
            login = mail_parts[0].lower().replace('.', '-')
            return login + '@' + domain
        else:
            return mail_.lower()


def works_with_phones(phone_):
    """phone processing
    :param phone_: phone (string or int)
    :return: processed phone as string (with only digits)
    """
    if not pd.isnull(phone_):
        phone_ = str(phone_)
        return ''.join(c for c in phone_ if c.isdigit())
    return phone_

        
def concatenate_tables(df_array):
    """Concatenates array of dataframes with possible Nan inside array
    :param df_array: array of dataframes
    :return: Nane or dataframe
    """

    df_array = [df for df in df_array if df is not None]
    if len(df_array) == 0:
        return None
    res_df = pd.concat(df_array, axis = 0)
    res_df.index = np.arange(0, res_df.shape[0], 1)
    return res_df


def apply_type(raw_schema, df):
    if raw_schema is not None:
        for key in raw_schema:
            if raw_schema[key] != 'list':
                df[key] = df[key].astype(raw_schema[key])
    schema = []
    for col in df.columns:
        if df[col].dtype == int:
            schema.append({"name":col, 'type':'int64'})
        elif df[col].dtype == float:
            schema.append({"name":col, 'type':'double'})
        elif raw_schema is not None and raw_schema.get(col) is not None and raw_schema[col] == 'list':
            schema.append({"name": col, 'type_v3':
                {"type_name": 'list', "item": {"type_name":"optional", "item":"string"}}})
        else:
            schema.append({"name":col, 'type':'string'})
    return schema

def save_table(file_to_write, path, table, schema = None, append=False):
    assert(path[-1] != '/')

    df = table.copy()
    real_schema = apply_type(schema, df)
    json_df_str = df.to_json(orient='records')
    path = path + "/" + file_to_write
    json_df = json.loads(json_df_str)
    if not yt.exists(path) or not append:
        yt.create(type="table", path=path, force=True,
              attributes={"schema": real_schema})
    tablepath = yt.TablePath(path, append=append)
    yt.write_table(tablepath, json_df,
               format = yt.JsonFormat(attributes={"encode_utf8": False}))

In [ ]:
def get_new_type(type_v3):
    if type_v3['type_name'] == 'list':
        return f"Array({get_new_type(type_v3['item'])})"
    else:
        return type_v3['item'][0].upper() + type_v3['item'][1:]


def create_schema_for_grafana(yt_path, table_name, sort_col=None):
    schema = yt.get(yt_path + "/@schema")

    body = f"CREATE TABLE {table_name} (\n    "
    time_cols = set()
    columns = set()
    int_cols = set()
    for key in schema:
        name = key["name"]
        columns.add(name)
        curr_type = get_new_type(key["type_v3"])
        if curr_type == "Datetime":
            time_cols.add(name)
        if curr_type == "Int64":
            int_cols.add(name)
        if key.get('sort_order') is not None and sort_col is None:
            sort_col = name
        body = body + name + " " + curr_type + ",\n    "

    body = body[:-6]
    body += "\n)\n"
    if len(time_cols) > 0:
        col = list(time_cols)[0]
    elif len(int_cols) > 0:
        col = list(int_cols)[0]
    else:
        col = list(columns)[0]
    if sort_col is None:
        sort_col = col
    partition_col = sort_col
    if sort_col in time_cols:
        partition_col = f"toYYYYMM({sort_col})"
    body += f"ENGINE = ReplicatedMergeTree('/clickhouse/tables/{{shard}}/{table_name}', '{{replica}}')\n" \
            f"ORDER BY {sort_col}\n"
    return body


def post_grafana_sql(query):
    hosts = ['sas-tt9078df91ipro7e.db.yandex.net',
             "vla-2z4ktcci90kq2bu2.db.yandex.net"]
    auth = {
        'X-ClickHouse-User': 'lunindv',
        'X-ClickHouse-Key': grafana_token,
    }
    for host in hosts:
        url = 'https://{host}:8443/?database={db}&query={query}'.format(
            host=host,
            db='cloud_analytics',
            query=query)
        r = requests.post(url=url,
                      headers=auth,
                      verify=\
                      '/usr/local/share/ca-certificates/Yandex/YandexInternalRootCA.crt')
        if r.status_code == 200:
            continue
        else:
            # print(host + r.text)
            pass
    return


def drop_grafana_table(grafana_table_name):
    try:
        post_grafana_sql('DROP TABLE ' + grafana_table_name)
    except:
        pass


def save_table_from_yt_to_grafana(yt_path, grafana_table_name,
                                  sort_col=None, schema_table=None):
    raw_grafana_table_name = grafana_table_name + "_" + str(int(datetime.now().timestamp()))
    if schema_table is None:
        schema_table=\
        create_schema_for_grafana(yt_path, raw_grafana_table_name, sort_col=sort_col)
    post_grafana_sql(schema_table)

    params = {
        'clickhouse_copy_options': {
            'command': 'append',
        },
        'clickhouse_credentials': {
            'password': grafana_token,
            'user': 'lunindv',
        },
        'mdb_auth': {
            'oauth_token': mdb_auth_token,
        },
        'mdb_cluster_address': {
            'cluster_id': "07bc5e8c-c4a7-4c26-b668-5a1503d858b9",
        },
        'clickhouse_copy_tool_settings_patch': {
            'clickhouse_client': {
                'per_shard_quorum': 'all',
            },
        }
    }
    task=tm.add_task(source_cluster="hahn",
            source_table=yt_path,
            destination_cluster='mdb-clickhouse',
            destination_table=raw_grafana_table_name,
            params=params,
            sync=False)
    task_info = tm.get_task_info(task)
    while task_info['state'] in ('pending', 'running'):
            time.sleep(5)
            task_info = tm.get_task_info(task)

    if task_info['state'] != 'completed':
            raise Exception(
                'Transfer manager task failed with '
                'the following state: %s' % task_info['state'])

    drop_grafana_table(grafana_table_name)
    move_req = f"""
    RENAME TABLE {raw_grafana_table_name}
        TO {grafana_table_name} ON CLUSTER "cloud_analytics"
    """
    post_grafana_sql(move_req)


In [ ]:
def get_current_date_as_str():
    return str(datetime.date(datetime.now()))

def date_to_string(date):
    return datetime.strftime(date, "%Y-%m-%d")

In [ ]:
SPARK_API = "http://webservicefarm.interfax.ru/IfaxWebService/ifaxwebservice.asmx"
params = {'Authmethod': "Login:yandex"}
body = requests.get(SPARK_API, auth=("yandex", "QZ0nEzj"))

In [ ]:
import zeep
wsdl = 'http://www.soapclient.com/xml/soapresponder.wsdl'
client = zeep.Client(wsdl=wsdl)
print(client.service.Method1('Zeep', 'is cool'))

In [ ]:
wsdl = "http://webservicefarm.interfax.ru/IfaxWebService/ifaxwebservice.asmx?WSDL"
client = zeep.Client(wsdl=wsdl)
print(client.service.Authmethod("yandex", "QZ0nEzj"))

In [ ]:
def get_dict(raw_result):
    # Черная json магия, сюда не лезть 
    req = str(raw_result).encode('utf-8')
    jsn_req = json.dumps(xmltodict.parse(req))
    return json.loads(jsn_req)["Response"]["Data"]

In [ ]:
def get_phone_str(phone):
    try:
        return "+7(" + phone["@Code"] + ")" + phone["@Number"]
    except Exception:
        pass
    return ""

In [ ]:
def get_phones(raw_res_GetCompanyShortReport):
    raw_res = raw_res_GetCompanyShortReport
    answer_list = []
    try:
        phones = get_dict(raw_res)["Report"]["PhoneList"]["Phone"]
    except Exception:
        return answer_list

    # Либо лист телефонов, либо словарь с одним телефоном
    if isinstance(phones, list):
        for phone in phones:
            answer_list.append(get_phone_str(phone))
    else:
        answer_list.append(get_phone_str(phones))
    return answer_list

In [ ]:
def get_leader_str(lieder):
    try:
        return lieder['@FIO'] + ", " + lieder['@Position']
    except Exception:
        pass
    try:
        return lieder['@FIO']
    except Exception:
        pass
    return ''

In [ ]:
def get_leader(raw_res_GetCompanyShortReport):
    res = get_dict(raw_res_GetCompanyShortReport)
    answer_list = []
    try:
        lieders = res["Report"]["LeaderList"]['Leader']
    except Exception:
        return answer_list
    # Либо лист руководителей, либо словарь с одним руководителем
    if isinstance(lieders, list):
        for lieder in lieders:
            answer_list.append(get_leader_str(lieder))            
    else:
        answer_list.append(get_leader_str(lieders))
    return answer_list

In [ ]:
def get_finance_results(raw_res_CompanyExtendedReport):
    raw_res = raw_res_CompanyExtendedReport
    finance_results = {}
    results = {}
    # Пройдемся по всей истории выручки и за каждый год сохраним выручку
    try:
        for fin_period in get_dict(raw_res)["Report"]["Finance"]['FinPeriod']:
            period = int(fin_period['@PeriodName'])
            mini_fin_results = fin_period['StringList']['String']
            for val in mini_fin_results:
                if val["@Name"] == 'Выручка':
                    finance_results[period] = int(val["@Value"])
    except Exception:
        pass
    
    # Если нет истории на последний год, то пусть будет доход в 1 рубль
    try:
        finance_results[last_year]
        # Сохраним выручку за последний год
        results[f"Revenue_{last_year}"] = finance_results[last_year]
    except Exception:
        finance_results[last_year] = 1
        results[f"Revenue_{last_year}"] = 0

    # Если нет истории на предпоследний последний год, 
    # то пусть будет доход, как в последний год, 
    # тогда тип выручки будет не растущим и не падающим
    try:
        finance_results[last_year - 1]
    except Exception:
        finance_results[last_year - 1] =  finance_results[last_year]
    
    # Посмотрим на рост прибыли
    results[f"Percentage_growth_from_{last_year - 1}"] =\
    (finance_results[last_year] - 
     finance_results[last_year - 1]) / finance_results[last_year - 1] * 100
    
    # Тип компании: растущая (рост выручки увеличился на 5 процентов)
    # убывающая (выручка упала на 5 процентов), иначе стабильно
    results["Revenue_type"] = "stable"
    if results[f"Percentage_growth_from_{last_year - 1}"] > 5:
        results["Revenue_type"] = "growth"
    if results[f"Percentage_growth_from_{last_year - 1}"] < -5:
        results["Revenue_type"] = "decline"
    return results

In [ ]:
def get_row_by_one_company(company):
    ans_dict = company.copy()
    spark_id = ans_dict['SparkID']
    raw_res_CompanyShortReport =\
        client.service.GetCompanyShortReport(spark_id, "", "")['xmlData']
    ans_dict["Phones_from_spark"] = get_phones(raw_res_CompanyShortReport)
    ans_dict["Supervisor"] = get_leader(raw_res_CompanyShortReport)
    
    raw_res_CompanyExtendedReport =\
    client.service.GetCompanyExtendedReport(spark_id)['xmlData']
    ans_dict = {**ans_dict,
                **get_finance_results(raw_res_CompanyExtendedReport)}
    return ans_dict

In [ ]:
def get_row_by_inn(inn):
    try:
        spark_id =\
        get_dict(client.service.CheckCompanyStatus('',inn, '')['xmlData'])['SparkID']
        raw_res_CompanyByCode =\
        client.service.FindCompanyByCode(inn, "", "0")['xmlData']
        companies = get_dict(raw_res_CompanyByCode)["Company"]
    except Exception:
        return None
    answer_list = []
    if isinstance(companies, list):
        for company in companies:
            if company["SparkID"] == spark_id:
                return get_row_by_one_company(company)
    else:
        return get_row_by_one_company(companies)

In [ ]:
def by_puid_get_inn_table():
    req = """
    SELECT
        passport_id as puid,
        argMax(inn, dt) as inn,
        argMax(name, dt) as name_for_cloud,
        argMax(phone, dt) as phone_for_cloud
    FROM "//home/cloud_analytics/import/balance/balance_persons"
    WHERE puid != 0
            AND isNotNull(puid)
    GROUP BY puid
    HAVING isNotNull(inn)
    FORMAT TabSeparatedWithNames
    """
    df = execute_query(req)
    return df

In [ ]:
last_year=2018

In [ ]:
df = by_puid_get_inn_table()

In [ ]:
df[df["puid"] == '1045911341']

In [ ]:
inns = []
for inn in df["inn"].unique():
    inns.append(inn)

In [ ]:
new_rows = []
for inn in tqdm(df["inn"].unique()):
    try:
        new_rows.append(get_row_by_inn(inn))
    except Exception:
        print(inn)
        pass

In [ ]:
all_rows = []
for arr in new_rows:
    curr_row = []
    if arr is not None: 
        curr_row = arr
        all_rows.append(curr_row)

In [ ]:
spark_df = pd.DataFrame(all_rows)

In [ ]:
spark_df.shape

In [ ]:
spark_df = pd.merge(spark_df, df, left_on="INN", right_on="inn", how='inner')

In [ ]:
spark_df.drop(columns=["inn"], inplace=True)

In [ ]:
spark_df.head()

In [ ]:
spark_df["phone_for_cloud"] = spark_df["phone_for_cloud"].apply(lambda x: works_with_phones(x))

In [ ]:
spark_df[spark_df["INN"] == '']

In [ ]:
save_table("spark_cube", 
           "//home/cloud_analytics/lunin-dv/spark", spark_df, schema={
           "Supervisor":"list", "Phones_from_spark": 'list'})

In [ ]:
save_table_from_yt_to_grafana("//home/cloud_analytics/lunin-dv/spark/spark_cube", 
                              "cloud_analytics.spark_cube", sort_col='SparkID')

In [ ]:
drop_grafana_table("spark_cube_1587059554")